In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::625209709701:role/SageMaker_Execution_Role_AWS


In [4]:
from sagemaker.sklearn import SKLearn

In [8]:
model = SKLearn(
    entry_point='script.py',
    source_dir='./',
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    framework_version='0.23-1',
    py_version='py3',
    output_path="s3://ml-project-demo-zakaria/model/",
    hyperparameters={
        'random_state': 42,
        'max_depth': 6,
        'n_estimators': 16
    },
    base_job_name='rf-training-job' 
)

In [9]:
raw_data_input = sagemaker.inputs.TrainingInput(
    s3_data='s3://ml-project-demo-zakaria/training_data/cleaned_data.csv',
    content_type='csv'
)

In [10]:
model.fit({'train': raw_data_input})

INFO:sagemaker:Creating training-job with name: rf-training-job-2025-07-24-19-04-11-535


2025-07-24 19:04:40 Starting - Starting the training job...
2025-07-24 19:05:04 Starting - Preparing the instances for training...
2025-07-24 19:05:27 Downloading - Downloading input data...
2025-07-24 19:05:47 Downloading - Downloading the training image...
2025-07-24 19:06:33 Training - Training image download completed. Training in progress..2025-07-24 19:06:45,030 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-07-24 19:06:45,034 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-24 19:06:45,081 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-07-24 19:06:46,535 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-24 19:06:46,548 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-24 19:06:46,561 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-07-24 19:

In [16]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name="first-test-3"
)

INFO:sagemaker:Creating model with name: rf-training-job-2025-07-24-19-26-31-349
INFO:sagemaker:Creating endpoint-config with name first-test-3
INFO:sagemaker:Creating endpoint with name first-test-3


-----!

In [21]:
s3_clean_traindata_bucket = "s3://ml-project-demo-zakaria/training_data/cleaned_data.csv"

In [40]:
import pandas as pd
import numpy as np

data = pd.read_csv(s3_clean_traindata_bucket)
data.head()

,IQ,Prev_Sem_Result,CGPA,Academic_Performance,Internship_Experience,Extra_Curricular_Score,Communication_Skills,Projects_Completed,Placement
0,107,6.61,6.28,8,0,8,8,4,No
1,97,5.52,5.37,8,0,7,8,0,No
2,109,5.36,5.83,9,0,3,1,1,No
3,122,5.47,5.75,6,1,1,6,1,No
4,96,7.91,7.69,7,0,8,10,2,No


In [51]:
test_data = data.drop('Placement', axis=1).sample(20).values.tolist()

In [52]:
test_data

[[100.0, 5.53, 6.02, 2.0, 0.0, 10.0, 10.0, 0.0],
 [88.0, 6.52, 6.66, 7.0, 1.0, 10.0, 9.0, 4.0],
 [88.0, 7.38, 7.02, 8.0, 1.0, 10.0, 7.0, 3.0],
 [124.0, 7.65, 7.73, 3.0, 0.0, 8.0, 9.0, 4.0],
 [95.0, 9.74, 9.89, 4.0, 0.0, 10.0, 9.0, 5.0],
 [101.0, 6.19, 6.43, 9.0, 1.0, 5.0, 4.0, 3.0],
 [107.0, 8.27, 8.56, 1.0, 1.0, 3.0, 2.0, 5.0],
 [101.0, 8.6, 8.57, 4.0, 0.0, 7.0, 4.0, 4.0],
 [114.0, 9.63, 9.6, 4.0, 0.0, 9.0, 4.0, 4.0],
 [130.0, 8.55, 8.5, 7.0, 0.0, 1.0, 4.0, 2.0],
 [96.0, 6.64, 6.9, 4.0, 1.0, 3.0, 1.0, 0.0],
 [123.0, 9.58, 9.08, 6.0, 1.0, 5.0, 3.0, 5.0],
 [66.0, 5.57, 5.16, 4.0, 0.0, 5.0, 5.0, 4.0],
 [94.0, 9.93, 9.89, 4.0, 0.0, 7.0, 6.0, 4.0],
 [105.0, 8.04, 8.3, 5.0, 1.0, 10.0, 2.0, 3.0],
 [120.0, 5.57, 5.33, 9.0, 0.0, 4.0, 9.0, 5.0],
 [108.0, 8.74, 9.0, 3.0, 1.0, 10.0, 9.0, 5.0],
 [104.0, 7.02, 7.1, 1.0, 1.0, 9.0, 2.0, 4.0],
 [91.0, 7.24, 7.15, 7.0, 1.0, 10.0, 7.0, 1.0],
 [101.0, 7.64, 7.77, 4.0, 1.0, 6.0, 9.0, 3.0]]

In [54]:
from sagemaker.deserializers import NumpyDeserializer

predictor.deserializer = NumpyDeserializer(allow_pickle=True)

result = predictor.predict(test_data)
print(result)

['No' 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No'
 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No']
